Clean dataset


In [ ]:
import pandas as pd
import openpyxl  # noqa: F401


def is_unnamed(header):
    return str(header).startswith("Unnamed")


def validate_first_sheet(df):
    return (
        is_unnamed(df.columns[1])
        and is_unnamed(df.columns[2])
        and is_unnamed(df.columns[3])
        and is_unnamed(df.columns[4])
        and is_unnamed(df.columns[5])
        and not pd.isna(df.columns[0])
        and not pd.isna(df.columns[6])
    )


def validate_other_sheet(df):
    return (
        df.shape[1] == 6
        and not is_unnamed(df.columns[1])
        and not is_unnamed(df.columns[2])
        and not is_unnamed(df.columns[3])
        and not is_unnamed(df.columns[4])
        and is_unnamed(df.columns[0])
    )


def process_files(articles_file_path, output_file_path):
    # Load the Excel file
    xls = pd.ExcelFile(articles_file_path)

    # Validate and load relevant sheets
    relevant_sheets = []
    first_sheet_validated = False

    for sheet_name in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet_name, header=0)
        if not first_sheet_validated:
            if validate_first_sheet(df):
                relevant_sheets.append((sheet_name, df))
                first_sheet_validated = True
            else:
                raise ValueError("First sheet is not valid")
        else:
            if validate_other_sheet(df):
                relevant_sheets.append((sheet_name, df))

    if not relevant_sheets:
        raise ValueError("No relevant sheets found in the Excel file")

    # Process the first sheet
    first_sheet_df = relevant_sheets[0][1].drop(columns=["mgs210"])

    # Align and merge other relevant sheets
    aligned_sheets = [first_sheet_df]
    for sheet_name, df in relevant_sheets[1:]:
        df.columns = first_sheet_df.columns
        aligned_sheets.append(df)

    # Concatenate all aligned sheets
    df_combined = pd.concat(aligned_sheets, ignore_index=True)

    # Drop rows where the third column (index 2) is empty or "."
    df_combined = df_combined[~df_combined.iloc[:, 2].isin(["", "."])]

    # Drop the "STAMPA LISTINI" column
    df_combined = df_combined.drop(columns=["STAMPA LISTINI"])

    # Print the number of columns for debugging
    print("Number of columns in df_combined:", len(df_combined.columns))
    print("Column names in df_combined:", df_combined.columns.tolist())

    # Rename the columns
    df_combined.columns = [
        "CODICE PRODOTTO",
        "BRAND",
        "DESCRIZIONE",
        "GIACENZA",
        "PRZ. ULT. ACQ.",
    ]

    # Add new columns
    df_combined["CODICE OE"] = pd.NA
    df_combined["CODICI CROSS"] = pd.NA
    df_combined["LINK IMMAGINE"] = pd.NA
    df_combined["CATEGORIA"] = pd.NA
    df_combined["SCHEDA TECNICA"] = pd.NA
    df_combined["SCHEDA DI SICUREZZA"] = pd.NA
    df_combined["CONFEZIONE"] = pd.NA
    df_combined["QUANTITÀ MINIMA"] = pd.NA
    df_combined["META.LUNGHEZZA"] = pd.NA
    df_combined["META.LARGHEZZA"] = pd.NA
    df_combined["META.PROFONDITA'"] = pd.NA
    df_combined["META. ..."] = pd.NA

    # Reorder the columns
    df_combined = df_combined[
        [
            "CODICE PRODOTTO",
            "CODICE OE",
            "CODICI CROSS",
            "BRAND",
            "DESCRIZIONE",
            "LINK IMMAGINE",
            "CATEGORIA",
            "PRZ. ULT. ACQ.",
            "GIACENZA",
            "SCHEDA TECNICA",
            "SCHEDA DI SICUREZZA",
            "CONFEZIONE",
            "QUANTITÀ MINIMA",
            "META.LUNGHEZZA",
            "META.LARGHEZZA",
            "META.PROFONDITA'",
            "META. ...",
        ]
    ]

    # Convert 'GIACENZA' and 'PRZ. ULT. ACQ.' columns to numeric types
    df_combined["GIACENZA"] = pd.to_numeric(
        df_combined["GIACENZA"].str.replace(",", "."), errors="coerce"
    )
    df_combined["PRZ. ULT. ACQ."] = pd.to_numeric(
        df_combined["PRZ. ULT. ACQ."].str.replace(",", "."), errors="coerce"
    )

    # Clean the data
    df_combined = df_combined[df_combined["GIACENZA"] > 0]
    df_combined = df_combined[df_combined["PRZ. ULT. ACQ."].notna()]

    # Save the cleaned data to a CSV file
    df_combined.to_csv(output_file_path, index=False)


# Placeholders for file paths
articles_file_path = "Z:/My Drive/rcs/oem cross/Articles26062024.xls"
output_file_path = "Z:/My Drive/rcs/oem cross/cleaned_Articles26062024.csv"

process_files(articles_file_path, output_file_path)


#MERGE OE CODES, BY APPENDING THEM#


In [ ]:
import pandas as pd


def aggregate_oem_numbers(oems_df):
    # Ensure 'oem_number' and 'article_alt' columns are treated as strings
    oems_df["oem_number"] = oems_df["oem_number"].astype(str)
    oems_df["article_alt"] = oems_df["article_alt"].astype(str)

    # Aggregate OEM numbers for each article_alt
    oems_agg = (
        oems_df.groupby("article_alt")["oem_number"]
        .apply(lambda x: " | ".join(x))
        .reset_index()
    )
    return oems_agg


def append_oem_numbers(df_cleaned, oems_agg):
    # Ensure 'CODICE PRODOTTO' is treated as a string
    df_cleaned["CODICE PRODOTTO"] = df_cleaned["CODICE PRODOTTO"].astype(str)

    # Strip leading/trailing spaces and convert to uppercase for both columns
    df_cleaned["CODICE PRODOTTO"] = (
        df_cleaned["CODICE PRODOTTO"].str.strip().str.upper()
    )
    oems_agg["article_alt"] = oems_agg["article_alt"].str.strip().str.upper()

    # Merge the cleaned dataframe with the aggregated OEM numbers
    merged_df = pd.merge(
        df_cleaned,
        oems_agg,
        left_on="CODICE PRODOTTO",
        right_on="article_alt",
        how="left",
    )

    # Rename the column for clarity
    merged_df.rename(columns={"oem_number": "CODICE OE"}, inplace=True)

    # Drop the article_alt column as it's no longer needed
    merged_df.drop(columns=["article_alt"], inplace=True)

    return merged_df


def process_and_merge_files(cleaned_csv_path, oems_file_path, output_file_path):
    # Load the cleaned articles CSV file
    df_cleaned = pd.read_csv(cleaned_csv_path)

    # Load the OEM CSV file with specified delimiter and as string type
    oems_df = pd.read_csv(oems_file_path, delimiter=";", dtype=str)

    # Aggregate the OEM numbers
    oems_agg = aggregate_oem_numbers(oems_df)

    # Append OEM numbers to the cleaned dataframe
    df_result = append_oem_numbers(df_cleaned, oems_agg)

    # Save the final dataframe to a CSV file
    df_result.to_csv(output_file_path, index=False)

    # Print some sample values for debugging
    print("Sample CODICE PRODOTTO values:")
    print(df_cleaned["CODICE PRODOTTO"].head(10))  # Print more rows for inspection
    print("Sample article_alt values:")
    print(oems_agg["article_alt"].head(10))  # Print more rows for inspection
    print("Sample merged CODICE OE values:")
    print(
        df_result[["CODICE PRODOTTO", "CODICE OE"]].head(10)
    )  # Print more rows for inspection

    # Print unique values in both columns for further inspection
    print("Unique CODICE PRODOTTO values:")
    print(df_cleaned["CODICE PRODOTTO"].unique()[:10])  # Print more unique values
    print("Unique article_alt values:")
    print(oems_agg["article_alt"].unique()[:10])  # Print more unique values


# Placeholders for file paths
articles_file_path = "Z:/My Drive/rcs/oem cross/cleaned_Articles26062024.csv"
oems_file_path = "Z:/My Drive/rcs/oem cross/oems.csv"
output_file_path = "Z:/My Drive/rcs/oem cross/OEMERGED_cleaned_Articles26062024.csv"

# Run the process
process_and_merge_files(articles_file_path, oems_file_path, output_file_path)

#SPLIT CSV FROM DT#


In [ ]:
import pandas as pd
import os
from tqdm import tqdm


def split_csv(input_csv_path, output_folder, output_csv_prefix, rows_per_file=1048575):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Count the total number of rows in the original CSV file
    total_rows = (
        sum(1 for row in open(input_csv_path, "r")) - 1
    )  # Subtract 1 for header row

    # Read the large CSV file in chunks
    chunk_iter = pd.read_csv(
        input_csv_path, chunksize=rows_per_file, delimiter=";", dtype=str
    )

    # Initialize file counter
    file_number = 1
    total_split_rows = 0

    for chunk in tqdm(chunk_iter, desc="Processing chunks"):
        # Drop columns at indices 5, 6, 7, and 8
        chunk.drop(chunk.columns[[0, 4, 5, 6, 7]], axis=1, inplace=True)

        # Count the rows in the current chunk
        chunk_rows = chunk.shape[0]
        total_split_rows += chunk_rows

        # Generate output file name with folder path
        output_csv_path = os.path.join(
            output_folder, f"{output_csv_prefix}_{file_number}.csv"
        )

        # Write chunk to a new CSV file
        chunk.to_csv(output_csv_path, index=False)

        # Increment file counter
        file_number += 1

    # Compare the total rows
    comparison_result = "Match" if total_rows == total_split_rows else "Mismatch"

    # Write the results to a text file
    with open(os.path.join(output_folder, "row_count_comparison.txt"), "w") as f:
        f.write(f"Total rows in original CSV: {total_rows}\n")
        f.write(f"Total rows in split CSVs: {total_split_rows}\n")
        f.write(f"Comparison result: {comparison_result}\n")


# Placeholders for file paths
input_csv_path = "Z:/My Drive/rcs/oem cross/oems.csv"
output_folder = "Z:/My Drive/rcs/oem cross/oemssplit/"
output_csv_prefix = "oemsDT"

# Run the split function
split_csv(input_csv_path, output_folder, output_csv_prefix, rows_per_file=1048575)


#SPLIT CSV FROM DC, BUT BRANDS UNIFIED WITH PRODUCT IDS#


In [ ]:
import pandas as pd
import os
from tqdm import tqdm


def split_csv(
    input_csv_path,
    output_folder,
    output_csv_prefix,
    rows_per_file=1048575,
    encoding="utf-8",
):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Count the total number of rows in the original CSV file
    with open(input_csv_path, "r", encoding=encoding) as f:
        total_rows = sum(1 for row in f) - 1  # Subtract 1 for header row

    # Read the large CSV file in chunks
    chunk_iter = pd.read_csv(
        input_csv_path,
        chunksize=rows_per_file,
        delimiter=";",
        dtype=str,
        encoding=encoding,
    )

    # Initialize file counter
    file_number = 1
    total_split_rows = 0

    for chunk in tqdm(chunk_iter, desc="Processing chunks"):
        # Drop columns at indices 5, 6, 7, and 8
        chunk.drop(chunk.columns[[0, 4, 5, 6, 7]], axis=1, inplace=True)

        # Count the rows in the current chunk
        chunk_rows = chunk.shape[0]
        total_split_rows += chunk_rows

        # Generate output file name with folder path
        output_csv_path = os.path.join(
            output_folder, f"{output_csv_prefix}_{file_number}.csv"
        )

        # Write chunk to a new CSV file
        chunk.to_csv(output_csv_path, index=False)

        # Increment file counter
        file_number += 1

    # Compare the total rows
    comparison_result = "Match" if total_rows == total_split_rows else "Mismatch"

    # Write the results to a text file
    with open(os.path.join(output_folder, "row_count_comparison.txt"), "w") as f:
        f.write(f"Total rows in original CSV: {total_rows}\n")
        f.write(f"Total rows in split CSVs: {total_split_rows}\n")
        f.write(f"Comparison result: {comparison_result}\n")


# Placeholders for file paths
input_csv_path = "Z:/My Drive/rcs/oem cross/oems_id.csv"
output_folder = "Z:/My Drive/rcs/oem cross/oemssplit/"
output_csv_prefix = "oemsDC"

# Run the split function
split_csv(
    input_csv_path,
    output_folder,
    output_csv_prefix,
    rows_per_file=1048575,
    encoding="utf-8",
)


#IDENTIFY BRANDS WITH MULTIPLE SPACES


In [ ]:
import pandas as pd
import os
from tqdm import tqdm


def identify_brands_with_spaces(
    input_folder, output_folder, brands_with_spaces_file, chunksize=100000
):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Initialize list for brands with spaces
    brands_with_spaces = []

    # List all CSV files in the input folder
    csv_files = [file for file in os.listdir(input_folder) if file.endswith(".csv")]

    # Iterate over all files in the input folder
    for file_name in tqdm(csv_files, desc="Identifying brands with spaces"):
        file_path = os.path.join(input_folder, file_name)

        # Process the file in chunks
        for chunk in pd.read_csv(file_path, dtype=str, chunksize=chunksize):
            # Ensure all values in the first column are strings
            chunk.iloc[:, 0] = chunk.iloc[:, 0].astype(str)

            # Strip leading/trailing spaces and identify rows with multiple spaces
            chunk["cleaned"] = chunk.iloc[:, 0].str.strip()
            mask = chunk["cleaned"].str.count(" ") > 1
            brands_with_spaces.extend(chunk[mask].values)

    # Save brands with spaces to a separate CSV file
    brands_with_spaces_df = pd.DataFrame(brands_with_spaces, columns=chunk.columns)
    brands_with_spaces_df.to_csv(
        os.path.join(output_folder, brands_with_spaces_file), index=False
    )

    print(f"Identified and saved brands with spaces to {brands_with_spaces_file}")


# Placeholders for file paths
input_folder = "Z:/My Drive/rcs/oem cross/oemssplit/"
output_folder = "Z:/My Drive/rcs/oem cross/oemssplit/"
brands_with_spaces_file = "brandswithspaces.csv"

# Identify brands with spaces and save them to a separate file
identify_brands_with_spaces(input_folder, output_folder, brands_with_spaces_file)


#IDENTIFY AND FIX BRANDS WITH SPACES#


In [ ]:
import pandas as pd
import os
from tqdm import tqdm


def apply_custom_rules(brand_product):
    if brand_product.startswith("SKF"):
        parts = brand_product.split(" ", 2)
        return f"{parts[0]} {parts[1]}{parts[2].replace(' ', '', 1)}"

    if brand_product.startswith("DIESEL POWER"):
        return brand_product.replace(" ", "-", 1)

    if brand_product.startswith("OPEN PARTS"):
        return brand_product.replace(" ", "-", 1)

    if brand_product.startswith("DT MT "):
        return brand_product.replace(" ", "-", 1)

    if brand_product.startswith("INCAR "):
        parts = brand_product.split(" ", 2)
        return f"{parts[0]} {parts[1]}{parts[2].replace(' ', '', 1)}"

    if brand_product.startswith("MERITOR"):
        parts = brand_product.split(" ", 3)
        return (
            f"{parts[0]} {parts[1]}{parts[2].replace(' ', '', 1).replace(' ', '', 1)}"
        )

    if brand_product.startswith("OE GERMANY"):
        parts = brand_product.split(" ", 2)
        return f"{parts[0]} {parts[1]}{parts[2].replace(' ', '', 1)}"

    if brand_product.startswith("RCS"):
        parts = brand_product.split(" ")
        if len(parts) == 2:
            return f"{parts[0]} {parts[1]}"
        elif len(parts) == 3:
            return f"{parts[0]} {parts[1]}{parts[2].replace(' ', '', 1)}"
        elif len(parts) >= 4:
            modified_part = parts[2].replace(" ", "", 1).replace(" ", "", 1)
            combined_part = f"{parts[1]}{modified_part}{parts[3]}"
            return f"{parts[0]} {combined_part} {' '.join(parts[4:])}"

    if brand_product.startswith("SEM LASTIK"):
        return brand_product.replace(" ", "-", 1)

    if brand_product.startswith("XXL TRUCK"):
        return brand_product.replace(" ", "-", 1)

    if brand_product.startswith("XXL  "):
        return brand_product.replace(" ", "", 1)

    return brand_product


def identify_and_apply_rules(
    input_folder, output_folder, brands_with_spaces_file, chunksize=100000
):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Initialize list for brands with spaces
    brands_with_spaces = []

    # List all CSV files in the input folder
    csv_files = [file for file in os.listdir(input_folder) if file.endswith(".csv")]

    # Iterate over all files in the input folder
    for file_name in tqdm(csv_files, desc="Processing files"):
        file_path = os.path.join(input_folder, file_name)

        # Process the file in chunks
        for chunk in pd.read_csv(file_path, dtype=str, chunksize=chunksize):
            # Ensure all values in the first column are strings
            chunk.iloc[:, 0] = chunk.iloc[:, 0].astype(str)

            # Strip leading/trailing spaces
            chunk.iloc[:, 0] = chunk.iloc[:, 0].str.strip()

            # Drop rows that start with "DT CAT" or "DT SA-FLYER EN"
            chunk = chunk[
                ~chunk.iloc[:, 0].str.startswith(("DT CAT", "DT SA-FLYER EN"))
            ]

            # Identify rows with multiple spaces
            mask = chunk.iloc[:, 0].str.count(" ") > 1

            # Apply custom rules to the rows identified
            for index in chunk[mask].index:
                brand_product = chunk.at[index, chunk.columns[0]]
                modified_brand_product = apply_custom_rules(brand_product)
                chunk.at[index, chunk.columns[0]] = modified_brand_product

            brands_with_spaces.extend(chunk[mask].values)

    # Save brands with spaces to a separate CSV file
    brands_with_spaces_df = pd.DataFrame(brands_with_spaces, columns=chunk.columns)
    brands_with_spaces_df.to_csv(
        os.path.join(output_folder, brands_with_spaces_file), index=False
    )

    print(f"Processed and saved brands with spaces to {brands_with_spaces_file}")


# Placeholders for file paths
input_folder = "Z:/My Drive/rcs/oem cross/oemssplit/"
output_folder = "Z:/My Drive/rcs/oem cross/oemssplit/"
brands_with_spaces_file = "brandswithspaces.csv"

# Identify brands with spaces, apply rules, and save them to a separate file
identify_and_apply_rules(input_folder, output_folder, brands_with_spaces_file)


#DIVIDE IDS AND BRANDS#


In [ ]:
import pandas as pd
import os
from tqdm import tqdm


def apply_custom_rules(brand_product):
    if brand_product.startswith("SKF"):
        parts = brand_product.split(" ", 2)
        return f"{parts[0]} {parts[1]}{parts[2].replace(' ', '', 1)}"

    if brand_product.startswith("DIESEL POWER"):
        return brand_product.replace(" ", "-", 1)

    if brand_product.startswith("OPEN PARTS"):
        return brand_product.replace(" ", "-", 1)

    if brand_product.startswith("DT MT "):
        return brand_product.replace(" ", "-", 1)

    if brand_product.startswith("INCAR "):
        parts = brand_product.split(" ", 2)
        return f"{parts[0]} {parts[1]}{parts[2].replace(' ', '', 1)}"

    if brand_product.startswith("MERITOR"):
        parts = brand_product.split(" ", 3)
        return (
            f"{parts[0]} {parts[1]}{parts[2].replace(' ', '', 1).replace(' ', '', 1)}"
        )

    if brand_product.startswith("OE GERMANY"):
        parts = brand_product.split(" ", 2)
        return f"{parts[0]} {parts[1]}{parts[2].replace(' ', '', 1)}"

    if brand_product.startswith("RCS"):
        parts = brand_product.split(" ")
        if len(parts) == 2:
            return f"{parts[0]} {parts[1]}"
        elif len(parts) == 3:
            return f"{parts[0]} {parts[1]}{parts[2].replace(' ', '', 1)}"
        elif len(parts) >= 4:
            modified_part = parts[2].replace(" ", "", 1).replace(" ", "", 1)
            combined_part = f"{parts[1]}{modified_part}{parts[3]}"
            return f"{parts[0]} {combined_part} {' '.join(parts[4:])}"

    if brand_product.startswith("SEM LASTIK"):
        return brand_product.replace(" ", "-", 1)

    if brand_product.startswith("XXL TRUCK"):
        return brand_product.replace(" ", "-", 1)

    if brand_product.startswith("XXL  "):
        return brand_product.replace(" ", "", 1)

    return brand_product


def process_csv_files(input_folder, output_folder, chunksize=100000):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # List all CSV files in the input folder
    csv_files = [file for file in os.listdir(input_folder) if file.endswith(".csv")]

    # Iterate over all files in the input folder
    for file_name in tqdm(csv_files, desc="Processing files"):
        file_path = os.path.join(input_folder, file_name)
        output_file_path = os.path.join(output_folder, file_name)

        processed_chunks = []

        # Process the file in chunks
        for chunk in pd.read_csv(file_path, dtype=str, chunksize=chunksize):
            # Ensure all values in the first column are strings
            chunk.iloc[:, 0] = chunk.iloc[:, 0].astype(str)

            # Strip leading/trailing spaces
            chunk.iloc[:, 0] = chunk.iloc[:, 0].str.strip()

            # Drop rows that start with "DT CAT" or "DT SA-FLYER EN"
            chunk = chunk[
                ~chunk.iloc[:, 0].str.startswith(("DT CAT", "DT SA-FLYER EN"))
            ]

            # Identify rows with multiple spaces
            mask = chunk.iloc[:, 0].str.count(" ") > 1

            # Apply custom rules to the rows identified
            for index in chunk[mask].index:
                brand_product = chunk.at[index, chunk.columns[0]]
                modified_brand_product = apply_custom_rules(brand_product)
                chunk.at[index, chunk.columns[0]] = modified_brand_product

            # Separate brand and product ID into new columns
            chunk["article_alt_brands"] = chunk.iloc[:, 0].str.extract(r"(^[^\s]+)")
            chunk["article_alt"] = chunk.iloc[:, 0].str.extract(r"^[^\s]+\s+(.+)")
            chunk["article_altc"] = chunk["article_alt"]
            chunk = chunk.drop(columns=["article_alt"])

            # Remove any spaces from the oem_number column
            chunk["oem_number"] = chunk["oem_number"].str.replace(" ", "")

            # Reorder columns to the required order
            chunk = chunk[
                ["article_altc", "article_alt_brands", "manufacturer", "oem_number"]
            ]

            processed_chunks.append(chunk)

        # Concatenate all processed chunks and save to a new CSV file
        processed_df = pd.concat(processed_chunks)
        processed_df.to_csv(output_file_path, index=False)

    print(f"Processed and saved files to {output_folder}")


# Placeholders for file paths
input_folder = "Z:/My Drive/rcs/oem cross/oemssplit/"
output_folder = "Z:/My Drive/rcs/oem cross/oemssplit/processed/"

# Process all CSV files to separate brands and product IDs
process_csv_files(input_folder, output_folder)


#FIX BRANDS NAMING#


In [ ]:
import pandas as pd
import os
from tqdm import tqdm

# Define file paths
oems_folder = "Z:/My Drive/rcs/oem cross/oemssplit/processed/"
output_folder = "Z:/My Drive/rcs/oem cross/oemssplit/processed/tulero/"
output_file = os.path.join(output_folder, "unique_brands.csv")

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Initialize a DataFrame to hold all OEM brands
all_oem_brands = pd.DataFrame(columns=["article_alt_brands"])

# List all OEM files in the processed folder
oems_files = [file for file in os.listdir(oems_folder) if file.endswith(".csv")]

# Iterate over all OEM files and collect brands
for file_name in tqdm(oems_files, desc="Collecting OEM brands"):
    file_path = os.path.join(oems_folder, file_name)
    oems_df = pd.read_csv(file_path, dtype=str)
    if "article_alt_brands" in oems_df.columns:
        all_oem_brands = pd.concat([all_oem_brands, oems_df[["article_alt_brands"]]])

# Drop duplicates to ensure unique brands
unique_brands = all_oem_brands.drop_duplicates().reset_index(drop=True)

# Save the result to a CSV file
unique_brands.to_csv(output_file, index=False)

print(f"Unique brands saved to {output_file}")


In [ ]:
import pandas as pd

# Define file paths
brands_to_check_file = (
    "Z:/My Drive/rcs/oem cross/oemssplit/processed/tulero/articlesbrands.csv"
)
existing_brands_file = (
    "Z:/My Drive/rcs/oem cross/oemssplit/processed/tulero/unique_brands.csv"
)
output_file = (
    "Z:/My Drive/rcs/oem cross/oemssplit/processed/tulero/unique_brands_result.csv"
)

# Load the CSV files
brands_to_check_df = pd.read_csv(brands_to_check_file, dtype=str)
existing_brands_df = pd.read_csv(existing_brands_file, dtype=str)

# Ensure the brand columns are stripped of leading/trailing spaces
brands_to_check_df["Brand"] = brands_to_check_df["Brand"].str.strip()
existing_brands_df["Brands"] = existing_brands_df["Brands"].str.strip()

# Create a set of existing brands for fast lookup
existing_brands_set = set(existing_brands_df["Brands"])

# Check for matches and add a new column with the matched brand or "No Match"
brands_to_check_df["Match"] = brands_to_check_df["Brand"].apply(
    lambda x: x if x in existing_brands_set else "No Match"
)

# Save the result to a new CSV file
brands_to_check_df.to_csv(output_file, index=False)

print(f"Output with matches saved to {output_file}")


#DO THE THING


In [ ]:
import pandas as pd
import os
from tqdm import tqdm

# Define file paths
cleaned_articles_file = "Z:/My Drive/rcs/oem cross/cleaned_Articles26062024.csv"
old_oems_folder = "Z:/My Drive/rcs/oem cross/oemssplit/processed/"
output_folder = "Z:/My Drive/rcs/oem cross/oemssplit/processed/tulero/"
log_file = os.path.join(output_folder, "match_log.txt")

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Load the cleaned articles file
cleaned_df = pd.read_csv(cleaned_articles_file, dtype=str)

# Ensure all values in the relevant columns are strings and strip any leading/trailing spaces
cleaned_df["CODICE PRODOTTO"] = cleaned_df["CODICE PRODOTTO"].astype(str).str.strip()
cleaned_df["CODICE OE"] = cleaned_df["CODICE OE"].astype(str).str.strip()
cleaned_df["BRAND"] = cleaned_df["BRAND"].astype(str).str.strip()

# List all old CSV files in the processed folder
old_oems_files = [
    file
    for file in os.listdir(old_oems_folder)
    if file.startswith("oemsDC") and file.endswith(".csv")
]

# Initialize a DataFrame to hold all OEM mappings
all_oem_mappings = pd.DataFrame()

# Iterate over all old oems files and build the oem_number mappings
for file_name in tqdm(old_oems_files, desc="Building old OEM mappings"):
    file_path = os.path.join(old_oems_folder, file_name)
    oems_df = pd.read_csv(file_path, dtype=str)
    oems_df["article_altc"] = oems_df["article_altc"].astype(str).str.strip()
    oems_df["oem_number"] = (
        oems_df["oem_number"].astype(str).str.strip().str.replace(" ", "")
    )
    oems_df["article_alt_brands"] = (
        oems_df["article_alt_brands"].astype(str).str.strip()
    )
    oems_df["brand_prefix"] = oems_df["article_alt_brands"].str[:5]
    all_oem_mappings = pd.concat(
        [all_oem_mappings, oems_df[["article_altc", "oem_number", "brand_prefix"]]]
    )

# Create a lookup dictionary
oem_lookup = (
    all_oem_mappings.groupby(["article_altc", "brand_prefix"])["oem_number"]
    .apply(list)
    .to_dict()
)


# Function to get OEM number
def get_oem_number(row):
    key = (row["CODICE PRODOTTO"], row["BRAND"][:5])
    return " | ".join(oem_lookup[key]) if key in oem_lookup else "Unknown OE"


# Update the CODICE OE column in the cleaned articles file
tqdm.pandas(desc="Updating CODICE OE with old OEMs")
cleaned_df["CODICE OE"] = cleaned_df.progress_apply(get_oem_number, axis=1)

# Increase the values in "PRZ. ULT. ACQ." by 25% and rename the column to "PREZZO"
cleaned_df["PRZ. ULT. ACQ."] = pd.to_numeric(
    cleaned_df["PRZ. ULT. ACQ."], errors="coerce"
)
cleaned_df["PREZZO"] = cleaned_df["PRZ. ULT. ACQ."].apply(
    lambda x: round(x * 1.25, 2) if pd.notnull(x) else x
)
cleaned_df = cleaned_df.drop(columns=["PRZ. ULT. ACQ."])

# Reorder the columns so that PREZZO is between GIACENZA and SCHEDA TECNICA
columns_order = [
    "CODICE PRODOTTO",
    "CODICE OE",
    "CODICI CROSS",
    "BRAND",
    "DESCRIZIONE",
    "LINK IMMAGINE",
    "CATEGORIA",
    "GIACENZA",
    "PREZZO",
    "SCHEDA TECNICA",
    "SCHEDA DI SICUREZZA",
    "CONFEZIONE",
    "QUANTITÀ MINIMA",
    "META.LUNGHEZZA",
    "META.LARGHEZZA",
    "META.PROFONDITA'",
    "META. ...",
]
cleaned_df = cleaned_df[columns_order]

# Initialize the CODICI CROSS column
cleaned_df["CODICI CROSS"] = ""

# Create a dictionary to hold cross references, excluding "Unknown OE"
cross_references = (
    cleaned_df[cleaned_df["CODICE OE"] != "Unknown OE"]
    .groupby("CODICE OE")["CODICE PRODOTTO"]
    .apply(list)
    .to_dict()
)

# Create a Series to store cross codes
cross_codes_series = pd.Series(index=cleaned_df.index, dtype=str)

# Apply the function to update the CODICI CROSS column, ignoring "Unknown OE"
for codice_oe, prodotti in tqdm(cross_references.items(), desc="Updating CODICI CROSS"):
    cross_codes = {
        prodotto: " | ".join([code for code in prodotti if code != prodotto])
        for prodotto in prodotti
    }
    cross_codes_series.update(pd.Series(cross_codes))

cleaned_df["CODICI CROSS"] = cleaned_df.index.map(cross_codes_series).fillna("")

# Precompute padded_oe column
cleaned_df["padded_oe"] = " " + cleaned_df["CODICE OE"].str.strip() + " "


# Handle Unknown OE: Check if CODICE PRODOTTO is present in other CODICE OE
def find_additional_cross_codes(codice_prodotto, padded_oe):
    matches = cleaned_df[cleaned_df["CODICE OE"] != "Unknown OE"]
    exact_matches = matches[
        matches["padded_oe"].str.contains(f" {codice_prodotto} ", regex=False)
    ]
    if not exact_matches.empty:
        return " | ".join(exact_matches["CODICE PRODOTTO"].unique())
    return ""


# Apply the function to update the CODICI CROSS column for Unknown OE
unknown_oe_mask = cleaned_df["CODICE OE"] == "Unknown OE"
cleaned_df.loc[unknown_oe_mask, "CODICI CROSS"] = cleaned_df.loc[
    unknown_oe_mask, "CODICE PRODOTTO"
].progress_apply(
    lambda codice_prodotto: find_additional_cross_codes(
        codice_prodotto, cleaned_df["padded_oe"]
    )
)

# Save the updated cleaned articles file
output_file_path = os.path.join(output_folder, "tulerodataset26062024.csv")
cleaned_df.to_csv(output_file_path, index=False)

# Write the match statistics to a log file
successful_matches = cleaned_df["CODICE OE"].apply(lambda x: x != "Unknown OE").sum()
unsuccessful_matches = cleaned_df["CODICE OE"].apply(lambda x: x == "Unknown OE").sum()

with open(log_file, "w") as log:
    log.write(f"Total successful matches: {successful_matches}\n")
    log.write(f"Total unsuccessful matches: {unsuccessful_matches}\n")

print(f"Updated CODICE OE and PREZZO in {output_file_path}")
print(f"Match statistics written to {log_file}")


#SAMPLE FILE#


In [1]:
import pandas as pd
import os

# Define file paths
input_file_path = "Z:/My Drive/rcs/Web/Tulero/output/final_output_with_brands.csv"
output_file_path = (
    "Z:/My Drive/rcs/Web/Tulero/output/sample_final_output_with_brands.csv"
)

# Load the dataset
df = pd.read_csv(input_file_path, dtype=str)

# Keep only the first occurrence of each unique BRAND
df_unique_brands = df.drop_duplicates(subset=["BRAND"], keep="first")

# Save the resulting dataframe to a new CSV file
df_unique_brands.to_csv(output_file_path, index=False)

print(f"Sample file with unique BRAND rows saved to {output_file_path}")


Sample file with unique BRAND rows saved to Z:/My Drive/rcs/Web/Tulero/output/sample_final_output_with_brands.csv


In [19]:
import pandas as pd
import openpyxl  # noqa: F401
import os
from tqdm import tqdm


def is_unnamed(header):
    return str(header).startswith("Unnamed")


def validate_first_sheet(df):
    return (
        is_unnamed(df.columns[1])
        and is_unnamed(df.columns[2])
        and is_unnamed(df.columns[3])
        and is_unnamed(df.columns[4])
        and is_unnamed(df.columns[5])
        and not pd.isna(df.columns[0])
        and not pd.isna(df.columns[6])
    )


def validate_other_sheet(df):
    return (
        df.shape[1] == 6
        and not is_unnamed(df.columns[1])
        and not is_unnamed(df.columns[2])
        and not is_unnamed(df.columns[3])
        and not is_unnamed(df.columns[4])
        and is_unnamed(df.columns[0])
    )


def process_files(articles_file_path, output_file_path):
    xls = pd.ExcelFile(articles_file_path)
    relevant_sheets = []
    first_sheet_validated = False

    for sheet_name in tqdm(xls.sheet_names, desc="Validating sheets"):
        df = pd.read_excel(xls, sheet_name=sheet_name, header=0)
        if not first_sheet_validated:
            if validate_first_sheet(df):
                relevant_sheets.append((sheet_name, df))
                first_sheet_validated = True
            else:
                raise ValueError("First sheet is not valid")
        else:
            if validate_other_sheet(df):
                relevant_sheets.append((sheet_name, df))

    if not relevant_sheets:
        raise ValueError("No relevant sheets found in the Excel file")

    first_sheet_df = relevant_sheets[0][1].drop(columns=["mgs210"])
    aligned_sheets = [first_sheet_df]
    for sheet_name, df in relevant_sheets[1:]:
        df.columns = first_sheet_df.columns
        aligned_sheets.append(df)

    df_combined = pd.concat(aligned_sheets, ignore_index=True)
    df_combined = df_combined[~df_combined.iloc[:, 2].isin(["", "."])]
    df_combined = df_combined.drop(columns=["STAMPA LISTINI"])

    df_combined.columns = [
        "CODICE PRODOTTO",
        "BRAND",
        "DESCRIZIONE",
        "GIACENZA",
        "PRZ. ULT. ACQ.",
    ]

    for col in [
        "CODICE OE",
        "CODICI CROSS",
        "LINK IMMAGINE",
        "CATEGORIA",
        "SCHEDA TECNICA",
        "SCHEDA DI SICUREZZA",
        "CONFEZIONE",
        "QUANTITÀ MINIMA",
        "META.LUNGHEZZA",
        "META.LARGHEZZA",
        "META.PROFONDITA'",
        "META. ...",
    ]:
        df_combined[col] = pd.NA

    df_combined = df_combined[
        [
            "CODICE PRODOTTO",
            "CODICE OE",
            "CODICI CROSS",
            "BRAND",
            "DESCRIZIONE",
            "LINK IMMAGINE",
            "CATEGORIA",
            "PRZ. ULT. ACQ.",
            "GIACENZA",
            "SCHEDA TECNICA",
            "SCHEDA DI SICUREZZA",
            "CONFEZIONE",
            "QUANTITÀ MINIMA",
            "META.LUNGHEZZA",
            "META.LARGHEZZA",
            "META.PROFONDITA'",
            "META. ...",
        ]
    ]

    df_combined["GIACENZA"] = pd.to_numeric(
        df_combined["GIACENZA"].str.replace(",", "."), errors="coerce"
    )
    df_combined["PRZ. ULT. ACQ."] = pd.to_numeric(
        df_combined["PRZ. ULT. ACQ."].str.replace(",", "."), errors="coerce"
    )

    df_combined = df_combined[df_combined["GIACENZA"] > 0]
    df_combined = df_combined[df_combined["PRZ. ULT. ACQ."].notna()]
    df_combined.to_csv(output_file_path, index=False)


def aggregate_oem_numbers(oems_df):
    oems_df["oem_number"] = oems_df["oem_number"].astype(str)
    oems_df["article_alt"] = oems_df["article_alt"].astype(str)
    oems_agg = (
        oems_df.groupby("article_alt")["oem_number"]
        .apply(lambda x: " | ".join(x))
        .reset_index()
    )
    return oems_agg


def append_oem_numbers(df_cleaned, oems_agg):
    df_cleaned["CODICE PRODOTTO"] = (
        df_cleaned["CODICE PRODOTTO"].astype(str).str.strip().str.upper()
    )
    oems_agg["article_alt"] = oems_agg["article_alt"].str.strip().str.upper()
    merged_df = pd.merge(
        df_cleaned,
        oems_agg,
        left_on="CODICE PRODOTTO",
        right_on="article_alt",
        how="left",
    )
    merged_df.rename(columns={"oem_number": "CODICE OE"}, inplace=True)
    merged_df.drop(columns=["article_alt"], inplace=True)
    return merged_df


def process_and_merge_files(cleaned_csv_path, oems_file_path, output_file_path):
    df_cleaned = pd.read_csv(cleaned_csv_path)
    oems_df = pd.read_csv(oems_file_path, delimiter=";", dtype=str)
    oems_agg = aggregate_oem_numbers(oems_df)
    df_result = append_oem_numbers(df_cleaned, oems_agg)
    df_result.to_csv(output_file_path, index=False)


def update_brands(output_file_path, brands_file_path, final_output_file_path):
    # Load the output file
    df_output = pd.read_csv(output_file_path, dtype=str)

    # Load the brands file
    brands_df = pd.read_csv(brands_file_path, dtype=str)

    # Ensure all values in the relevant columns are strings and strip any leading/trailing spaces
    df_output["BRAND"] = df_output["BRAND"].astype(str).str.strip()
    brands_df["Brand"] = brands_df["Brand"].astype(str).str.strip()
    brands_df["Match"] = brands_df["Match"].astype(str).str.strip()

    # Create a lookup dictionary from the brands file
    brand_lookup = dict(zip(brands_df["Brand"], brands_df["Match"]))

    # Update the BRANDS in the output dataframe
    tqdm.pandas(desc="Updating brands")
    df_output["BRAND"] = df_output["BRAND"].progress_apply(
        lambda x: brand_lookup.get(x, x)
    )

    # Save the updated dataframe to a new CSV file
    df_output.to_csv(final_output_file_path, index=False)


def main():
    # Define file paths
    articles_file_path = "Z:/My Drive/rcs/oem cross/Articles26062024.xls"
    cleaned_csv_path = "Z:/My Drive/rcs/oem cross/cleaned_Articles26062024.csv"
    old_oems_folder = "Z:/My Drive/rcs/oem cross/oemssplit/processed/"
    output_folder = "Z:/My Drive/rcs/oem cross/oemssplit/processed/tulero/"
    final_output_file_path = os.path.join(output_folder, "tulerodataset26062024.csv")
    final_output_with_brands_path = os.path.join(
        output_folder, "tulerodataset_with_brands26062024.csv"
    )
    log_file = os.path.join(output_folder, "match_log.txt")
    brands_file_path = "Z:/My Drive/rcs/oem cross/BRANDS.csv"

    os.makedirs(output_folder, exist_ok=True)

    # Step 1: Clean dataset
    process_files(articles_file_path, cleaned_csv_path)

    # Step 2: Do the thing
    cleaned_df = pd.read_csv(cleaned_csv_path, dtype=str)
    cleaned_df["CODICE PRODOTTO"] = (
        cleaned_df["CODICE PRODOTTO"].astype(str).str.strip()
    )
    cleaned_df["CODICE OE"] = cleaned_df["CODICE OE"].astype(str).str.strip()
    cleaned_df["BRAND"] = cleaned_df["BRAND"].astype(str).str.strip()

    old_oems_files = [
        file
        for file in os.listdir(old_oems_folder)
        if file.startswith("oemsDC") and file.endswith(".csv")
    ]
    all_oem_mappings = pd.DataFrame()

    for file_name in tqdm(old_oems_files, desc="Building old OEM mappings"):
        file_path = os.path.join(old_oems_folder, file_name)
        oems_df = pd.read_csv(file_path, dtype=str)
        oems_df["article_altc"] = oems_df["article_altc"].astype(str).str.strip()
        oems_df["oem_number"] = (
            oems_df["oem_number"].astype(str).str.strip().str.replace(" ", "")
        )
        oems_df["article_alt_brands"] = (
            oems_df["article_alt_brands"].astype(str).str.strip()
        )
        oems_df["brand_prefix"] = oems_df["article_alt_brands"].str[:5]
        all_oem_mappings = pd.concat(
            [all_oem_mappings, oems_df[["article_altc", "oem_number", "brand_prefix"]]]
        )

    oem_lookup = (
        all_oem_mappings.groupby(["article_altc", "brand_prefix"])["oem_number"]
        .apply(list)
        .to_dict()
    )

    def get_oem_number(row):
        key = (row["CODICE PRODOTTO"], row["BRAND"][:5])
        return " | ".join(oem_lookup[key]) if key in oem_lookup else "Unknown OE"

    tqdm.pandas(desc="Updating CODICE OE with old OEMs")
    cleaned_df["CODICE OE"] = cleaned_df.progress_apply(get_oem_number, axis=1)

    cleaned_df["PRZ. ULT. ACQ."] = pd.to_numeric(
        cleaned_df["PRZ. ULT. ACQ."], errors="coerce"
    )
    cleaned_df["PREZZO"] = cleaned_df["PRZ. ULT. ACQ."].apply(
        lambda x: round(x * 1.25, 2) if pd.notnull(x) else x
    )
    cleaned_df = cleaned_df.drop(columns=["PRZ. ULT. ACQ."])

    columns_order = [
        "CODICE PRODOTTO",
        "CODICE OE",
        "CODICI CROSS",
        "BRAND",
        "DESCRIZIONE",
        "LINK IMMAGINE",
        "CATEGORIA",
        "GIACENZA",
        "PREZZO",
        "SCHEDA TECNICA",
        "SCHEDA DI SICUREZZA",
        "CONFEZIONE",
        "QUANTITÀ MINIMA",
        "META.LUNGHEZZA",
        "META.LARGHEZZA",
        "META.PROFONDITA'",
        "META. ...",
    ]
    cleaned_df = cleaned_df[columns_order]
    cleaned_df["CODICI CROSS"] = ""

    cross_references = (
        cleaned_df[cleaned_df["CODICE OE"] != "Unknown OE"]
        .groupby("CODICE OE")["CODICE PRODOTTO"]
        .apply(list)
        .to_dict()
    )
    cross_codes_series = pd.Series(index=cleaned_df.index, dtype=str)

    for codice_oe, prodotti in tqdm(
        cross_references.items(), desc="Updating CODICI CROSS"
    ):
        cross_codes = {
            prodotto: " | ".join([code for code in prodotti if code != prodotto])
            for prodotto in prodotti
        }
        cross_codes_series.update(pd.Series(cross_codes))

    cleaned_df["CODICI CROSS"] = cleaned_df.index.map(cross_codes_series).fillna("")
    cleaned_df["padded_oe"] = " " + cleaned_df["CODICE OE"].str.strip() + " "

    def find_additional_cross_codes(codice_prodotto, padded_oe):
        matches = cleaned_df[cleaned_df["CODICE OE"] != "Unknown OE"]
        exact_matches = matches[
            matches["padded_oe"].str.contains(f" {codice_prodotto} ", regex=False)
        ]
        if not exact_matches.empty:
            return " | ".join(exact_matches["CODICE PRODOTTO"].unique())
        return ""

    unknown_oe_mask = cleaned_df["CODICE OE"] == "Unknown OE"
    cleaned_df.loc[unknown_oe_mask, "CODICI CROSS"] = cleaned_df.loc[
        unknown_oe_mask, "CODICE PRODOTTO"
    ].progress_apply(
        lambda codice_prodotto: find_additional_cross_codes(
            codice_prodotto, cleaned_df["padded_oe"]
        )
    )

    cleaned_df.to_csv(final_output_file_path, index=False)

    successful_matches = (
        cleaned_df["CODICE OE"].apply(lambda x: x != "Unknown OE").sum()
    )
    unsuccessful_matches = (
        cleaned_df["CODICE OE"].apply(lambda x: x == "Unknown OE").sum()
    )

    with open(log_file, "w") as log:
        log.write(f"Total successful matches: {successful_matches}\n")
        log.write(f"Total unsuccessful matches: {unsuccessful_matches}\n")

    print(f"Updated CODICE OE and PREZZO in {final_output_file_path}")
    print(f"Match statistics written to {log_file}")

    # Step 4: Update brands
    update_brands(
        final_output_file_path, brands_file_path, final_output_with_brands_path
    )
    print(f"Brands updated in {final_output_with_brands_path}")


if __name__ == "__main__":
    main()


Updating CODICE OE with old OEMs: 100%|██████████| 13842/13842 [01:15<00:00, 182.16it/s]


Updated CODICE OE and PREZZO in Z:/My Drive/rcs/oem cross/oemssplit/processed/tulero/tulerodataset26062024.csv
Match statistics written to Z:/My Drive/rcs/oem cross/oemssplit/processed/tulero/match_log.txt


Updating brands: 100%|██████████| 30376/30376 [00:00<00:00, 3038037.49it/s]

Brands updated in Z:/My Drive/rcs/oem cross/oemssplit/processed/tulero/tulerodataset_with_brands26062024.csv


In [1]:
import pandas as pd
import os
from tqdm import tqdm

IGNORED_BRANDS = [
    "AP",
    "AREXONS",
    "ASSO",
    "ATE",
    "ATECSO",
    "AUTOCLIMA",
    "BIRTH",
    "BOSCH",
    "BREMBO",
    "BUGATTI",
    "CASCO",
    "CASTROL",
    "CEI",
    "CORTECO",
    "COVIND",
    "DAF",
    "DAYCO",
    "DELPHI",
    "DENSO",
    "DOLZ",
    "ELRING",
    "EMMERRE",
    "ERA",
    "EXIDE",
    "FAG",
    "FEBI",
    "FERODO",
    "FIAT",
    "FORD",
    "FRAP",
    "FTE",
    "GATES",
    "HELLA",
    "HOFFER",
    "IMASAF",
    "INA",
    "ISUZU",
    "IVECO",
    "JAPANPARTS",
    "JAPKO",
    "KNECHT",
    "KRIOS",
    "LEMFORDER",
    "LUK",
    "MAHLE",
    "MAN",
    "METELLI",
    "MEYLE",
    "MOBIL",
    "MONROE",
    "MOOG",
    "MULLER FILTER",
    "NISSAN",
    "NISSENS",
    "NK",
    "NRF",
    "OLSA",
    "OMP",
    "PEUGEOT",
    "PIAGGIO",
    "PIERBURG",
    "RAICAM",
    "RENAULT",
    "SACHS",
    "SCANIA",
    "SELENIA",
    "SIDAT",
    "SKF",
    "TEXTAR",
    "TRW",
    "TUDOR",
    "UFI",
    "VALEO",
    "VEMA",
    "VITAL SUSPENSIONS",
    "VOLVO",
    "VOLKSWAGEN",
    "ZETA-ERRE",
    "ZF",
]


def is_unnamed(header):
    return str(header).startswith("Unnamed")


def validate_first_sheet(df):
    return (
        is_unnamed(df.columns[1])
        and is_unnamed(df.columns[2])
        and is_unnamed(df.columns[3])
        and is_unnamed(df.columns[4])
        and is_unnamed(df.columns[5])
        and not pd.isna(df.columns[0])
        and not pd.isna(df.columns[6])
    )


def validate_other_sheet(df):
    return (
        df.shape[1] == 6
        and not is_unnamed(df.columns[1])
        and not is_unnamed(df.columns[2])
        and not is_unnamed(df.columns[3])
        and not is_unnamed(df.columns[4])
        and is_unnamed(df.columns[0])
    )


def get_oem_number(row, oem_lookup, IGNORED_BRANDS):
    if row["BRAND"] in IGNORED_BRANDS:
        return ""  # Skip filling OE code for ignored brands
    key = (row["CODICE PRODOTTO"], row["BRAND"][:5])
    return " | ".join(oem_lookup[key]) if key in oem_lookup else "Unknown OE"


def find_additional_cross_codes(codice_prodotto, padded_oe, cleaned_df, IGNORED_BRANDS):
    matches = cleaned_df[
        (cleaned_df["CODICE OE"] != "Unknown OE")
        & (~cleaned_df["BRAND"].isin(IGNORED_BRANDS))
    ]
    exact_matches = matches[
        matches["padded_oe"].str.contains(f" {codice_prodotto} ", regex=False)
    ]
    if not exact_matches.empty:
        return " | ".join(exact_matches["CODICE PRODOTTO"].unique())
    return ""


def update_brands(df_output, brands_file_path):
    # Load the brands file
    brands_df = pd.read_csv(brands_file_path, dtype=str)
    brands_df["Brand"] = brands_df["Brand"].astype(str).str.strip()
    brands_df["Match"] = brands_df["Match"].astype(str).str.strip()

    # Create a lookup dictionary from the brands file
    brand_lookup = dict(zip(brands_df["Brand"], brands_df["Match"]))

    # Update the BRANDS in the output dataframe
    tqdm.pandas(desc="Updating brands")
    df_output["BRAND"] = df_output["BRAND"].progress_apply(
        lambda x: brand_lookup.get(x, x)
    )
    return df_output


def optimized_cross_code_generation(cleaned_df, ignored_brands):
    cross_references = {}
    for _, row in tqdm(
        cleaned_df.iterrows(), total=len(cleaned_df), desc="Generating cross codes"
    ):
        codice_oe = row["CODICE OE"]
        codice_prodotto = row["CODICE PRODOTTO"]
        brand = row["BRAND"]

        if codice_oe != "Unknown OE" and brand not in ignored_brands:
            if codice_oe not in cross_references:
                cross_references[codice_oe] = []
            cross_references[codice_oe].append(codice_prodotto)

    cross_codes_series = pd.Series(index=cleaned_df.index, dtype=str)
    for codice_oe, prodotti in tqdm(
        cross_references.items(), desc="Updating CODICI CROSS"
    ):
        cross_codes = {
            prodotto: " | ".join([code for code in prodotti if code != prodotto])
            for prodotto in prodotti
        }
        cross_codes_series.update(pd.Series(cross_codes))

    return cross_codes_series.fillna("")


def load_and_clean_warehouse_file(
    warehouse_location_file, warehouse_unknown_location_path
):
    xls = pd.ExcelFile(warehouse_location_file)
    relevant_sheets = []
    first_sheet_validated = False

    for sheet_name in tqdm(xls.sheet_names, desc="Validating sheets"):
        df = pd.read_excel(xls, sheet_name=sheet_name, header=0, dtype=str)
        if not first_sheet_validated:
            if validate_first_sheet(df):
                relevant_sheets.append((sheet_name, df))
                first_sheet_validated = True
            else:
                raise ValueError("First sheet is not valid")
        else:
            if validate_other_sheet(df):
                relevant_sheets.append((sheet_name, df))

    if not relevant_sheets:
        raise ValueError("No relevant sheets found in the Excel file")

    # Identify and drop the column that starts with 'mgs'
    first_sheet_df = relevant_sheets[0][1]
    mgs_column = [col for col in first_sheet_df.columns if col.startswith("mgs")]
    if mgs_column:
        first_sheet_df = first_sheet_df.drop(columns=mgs_column)
    else:
        raise ValueError("No column starting with 'mgs' found")

    aligned_sheets = [first_sheet_df]
    for sheet_name, df in relevant_sheets[1:]:
        df.columns = first_sheet_df.columns
        aligned_sheets.append(df)

    df_combined = pd.concat(aligned_sheets, ignore_index=True)
    df_combined = df_combined[~df_combined.iloc[:, 2].isin(["", "."])]

    # Drop the 'STAMPA ANAGRAFICA ARTICOLI' column
    df_combined = df_combined.drop(columns=["STAMPA ANAGRAFICA ARTICOLI"])

    # Check the number of columns before renaming
    print(f"Number of columns before renaming: {len(df_combined.columns)}")
    print(f"Column names before renaming: {df_combined.columns.tolist()}")

    # Assign column names based on the number of columns
    if len(df_combined.columns) == 5:
        df_combined.columns = [
            "CODICE PRODOTTO",
            "BRAND",
            "DESCRIZIONE",
            "UBICAZIONE",
            "GIACENZA",
        ]
    else:
        raise ValueError(f"Unexpected number of columns: {len(df_combined.columns)}")

    df_combined["UBICAZIONE"] = (
        df_combined["UBICAZIONE"].astype(str).replace("", "Location Unknown")
    )

    # Cleaning UBICAZIONE column
    pattern = r"^[A-Z]\."  # Regex pattern to match a letter followed by a "."
    valid_rows = df_combined["UBICAZIONE"].str.match(pattern)
    valid_df = df_combined[valid_rows]
    invalid_df = df_combined[~valid_rows]

    # Output the rows with unknown locations
    invalid_df.to_csv(warehouse_unknown_location_path, index=False)

    return valid_df


def merge_with_warehouse_data(cleaned_df, warehouse_df):
    # Merging based on CODICE PRODOTTO and BRAND
    merged_df = pd.merge(
        cleaned_df,
        warehouse_df[["CODICE PRODOTTO", "BRAND", "UBICAZIONE"]],
        on=["CODICE PRODOTTO", "BRAND"],
        how="left",
    )

    # Fill NaN locations with empty string to avoid issues later
    merged_df["UBICAZIONE"] = merged_df["UBICAZIONE"].fillna("")

    return merged_df


def process_files(
    articles_file_path,
    output_file_path,
    old_oems_folder,
    brands_file_path,
    final_output_with_brands_path,
    update_progress=None,
):
    xls = pd.ExcelFile(articles_file_path)
    relevant_sheets = []
    first_sheet_validated = False

    for sheet_name in tqdm(xls.sheet_names, desc="Validating sheets"):
        df = pd.read_excel(
            xls, sheet_name=sheet_name, header=0, dtype=str
        )  # Ensure all columns are read as strings
        if not first_sheet_validated:
            if validate_first_sheet(df):
                relevant_sheets.append((sheet_name, df))
                first_sheet_validated = True
            else:
                raise ValueError("First sheet is not valid")
        else:
            if validate_other_sheet(df):
                relevant_sheets.append((sheet_name, df))

    if not relevant_sheets:
        raise ValueError("No relevant sheets found in the Excel file")

    first_sheet_df = relevant_sheets[0][1].drop(columns=["mgs210"])
    aligned_sheets = [first_sheet_df]
    for sheet_name, df in relevant_sheets[1:]:
        df.columns = first_sheet_df.columns
        aligned_sheets.append(df)

    df_combined = pd.concat(aligned_sheets, ignore_index=True)
    df_combined = df_combined[~df_combined.iloc[:, 2].isin(["", "."])]
    df_combined = df_combined.drop(columns=["STAMPA LISTINI"])

    df_combined.columns = [
        "CODICE PRODOTTO",
        "BRAND",
        "DESCRIZIONE",
        "GIACENZA",
        "PRZ. ULT. ACQ.",
    ]

    for col in [
        "CODICE OE",
        "CODICI CROSS",
        "LINK IMMAGINE",
        "CATEGORIA",
        "SCHEDA TECNICA",
        "SCHEDA DI SICUREZZA",
        "CONFEZIONE",
        "QUANTITÀ MINIMA",
        "META.LUNGHEZZA",
        "META.LARGHEZZA",
        "META.PROFONDITA'",
        "META. ...",
    ]:
        df_combined[col] = pd.NA

    # Ensure specified columns are treated as strings
    columns_to_ensure_as_strings = [
        "CODICE PRODOTTO",
        "BRAND",
        "DESCRIZIONE",
        "CODICE OE",
        "CODICI CROSS",
        "LINK IMMAGINE",
        "CATEGORIA",
        "SCHEDA TECNICA",
        "SCHEDA DI SICUREZZA",
        "CONFEZIONE",
        "META.LUNGHEZZA",
        "META.LARGHEZZA",
        "META.PROFONDITA'",
        "META. ...",
    ]
    df_combined[columns_to_ensure_as_strings] = df_combined[
        columns_to_ensure_as_strings
    ].astype(str)

    df_combined["GIACENZA"] = pd.to_numeric(
        df_combined["GIACENZA"].str.replace(",", "."), errors="coerce"
    )
    df_combined["PRZ. ULT. ACQ."] = pd.to_numeric(
        df_combined["PRZ. ULT. ACQ."].str.replace(",", "."), errors="coerce"
    )

    df_combined = df_combined[df_combined["GIACENZA"] > 0]
    df_combined = df_combined[df_combined["PRZ. ULT. ACQ."].notna()]

    # No need to save and reload, continue with the DataFrame in memory
    cleaned_df = df_combined.copy()
    cleaned_df["CODICE PRODOTTO"] = cleaned_df["CODICE PRODOTTO"].str.strip()
    cleaned_df["CODICE OE"] = cleaned_df["CODICE OE"].str.strip()
    cleaned_df["BRAND"] = cleaned_df["BRAND"].str.strip()

    old_oems_files = [
        file
        for file in os.listdir(old_oems_folder)
        if file.startswith("oemsDC") and file.endswith(".csv")
    ]
    all_oem_mappings = pd.DataFrame()

    for file_name in tqdm(old_oems_files, desc="Building old OEM mappings"):
        file_path = os.path.join(old_oems_folder, file_name)
        oems_df = pd.read_csv(file_path, dtype=str)
        oems_df["article_altc"] = oems_df["article_altc"].astype(str).str.strip()
        oems_df["oem_number"] = (
            oems_df["oem_number"].astype(str).str.strip().str.replace(" ", "")
        )
        oems_df["article_alt_brands"] = (
            oems_df["article_alt_brands"].astype(str).str.strip()
        )
        oems_df["brand_prefix"] = oems_df["article_alt_brands"].str[:5]
        all_oem_mappings = pd.concat(
            [all_oem_mappings, oems_df[["article_altc", "oem_number", "brand_prefix"]]]
        )

    oem_lookup = (
        all_oem_mappings.groupby(["article_altc", "brand_prefix"])["oem_number"]
        .apply(list)
        .to_dict()
    )

    # Update CODICE OE using the get_oem_number function
    tqdm.pandas(desc="Updating CODICE OE with old OEMs")
    cleaned_df["CODICE OE"] = cleaned_df.progress_apply(
        lambda row: get_oem_number(row, oem_lookup, IGNORED_BRANDS), axis=1
    )

    # Update PREZZO based on PRZ. ULT. ACQ.
    cleaned_df["PRZ. ULT. ACQ."] = pd.to_numeric(
        cleaned_df["PRZ. ULT. ACQ."], errors="coerce"
    )
    cleaned_df["PREZZO"] = cleaned_df["PRZ. ULT. ACQ."].apply(
        lambda x: round(x * 1.25, 2) if pd.notnull(x) else x
    )
    cleaned_df = cleaned_df.drop(columns=["PRZ. ULT. ACQ."])

    # Ensure specified columns are treated as strings before reordering
    cleaned_df[columns_to_ensure_as_strings] = cleaned_df[
        columns_to_ensure_as_strings
    ].astype(str)

    # Reorder columns
    columns_order = [
        "CODICE PRODOTTO",
        "CODICE OE",
        "CODICI CROSS",
        "BRAND",
        "DESCRIZIONE",
        "LINK IMMAGINE",
        "CATEGORIA",
        "GIACENZA",
        "PREZZO",
        "SCHEDA TECNICA",
        "SCHEDA DI SICUREZZA",
        "CONFEZIONE",
        "QUANTITÀ MINIMA",
        "META.LUNGHEZZA",
        "META.LARGHEZZA",
        "META.PROFONDITA'",
        "META. ...",
    ]
    cleaned_df = cleaned_df[columns_order]
    cleaned_df["CODICI CROSS"] = ""

    # Apply optimized cross-code generation function
    cleaned_df["CODICI CROSS"] = optimized_cross_code_generation(
        cleaned_df, IGNORED_BRANDS
    )

    # Handle cases where CODICE OE is unknown and brand is not ignored
    cleaned_df["padded_oe"] = " " + cleaned_df["CODICE OE"].str.strip() + " "
    unknown_oe_mask = (cleaned_df["CODICE OE"] == "Unknown OE") & (
        ~cleaned_df["BRAND"].isin(IGNORED_BRANDS)
    )
    cleaned_df.loc[unknown_oe_mask, "CODICI CROSS"] = cleaned_df.loc[
        unknown_oe_mask, "CODICE PRODOTTO"
    ].progress_apply(
        lambda codice_prodotto: find_additional_cross_codes(
            codice_prodotto, cleaned_df["padded_oe"], cleaned_df, IGNORED_BRANDS
        )
    )

    # Drop the 'padded_oe' column
    cleaned_df = cleaned_df.drop(columns=["padded_oe"])

    # Fill the "CONFEZIONE" column with "1 pz" and the "QUANTITÀ MINIMA" column with "1"
    cleaned_df["CONFEZIONE"] = "1 pz"
    cleaned_df["QUANTITÀ MINIMA"] = "1"

    # Set CATEGORIA to "Ricambio"
    cleaned_df["CATEGORIA"] = "Ricambio"

    # Ensure specified columns remain empty
    columns_to_keep_empty = [
        "LINK IMMAGINE",
        "SCHEDA TECNICA",
        "SCHEDA DI SICUREZZA",
        "META.LUNGHEZZA",
        "META.LARGHEZZA",
        "META.PROFONDITA'",
        "META. ...",
    ]
    cleaned_df[columns_to_keep_empty] = ""

    return cleaned_df


def process_files_with_location(
    articles_file_path,
    output_file_path,
    old_oems_folder,
    brands_file_path,
    warehouse_location_file,
    final_output_with_brands_path,
    output_csv_small_items_path,
    output_csv_large_items_path,
    warehouse_unknown_location_path,
    update_progress=None,
):
    # Load and clean the articles file (same as original)
    cleaned_df = process_files(
        articles_file_path,
        output_file_path,
        old_oems_folder,
        brands_file_path,
        final_output_with_brands_path,
        update_progress,
    )

    # Load and clean the warehouse location file
    warehouse_df = load_and_clean_warehouse_file(
        warehouse_location_file, warehouse_unknown_location_path
    )

    # Merge the warehouse location data with the cleaned data
    cleaned_df = merge_with_warehouse_data(cleaned_df, warehouse_df)

    # Update brands as before
    cleaned_df = update_brands(cleaned_df, brands_file_path)

    # Final reordering of columns
    columns_order = [
        "CODICE PRODOTTO",
        "CODICE OE",
        "CODICI CROSS",
        "BRAND",
        "DESCRIZIONE",
        "LINK IMMAGINE",
        "CATEGORIA",
        "UBICAZIONE",
        "GIACENZA",
        "PREZZO",
        "SCHEDA TECNICA",
        "SCHEDA DI SICUREZZA",
        "CONFEZIONE",
        "QUANTITÀ MINIMA",
        "META.LUNGHEZZA",
        "META.LARGHEZZA",
        "META.PROFONDITA'",
        "META. ...",
    ]
    cleaned_df = cleaned_df[columns_order]

    # Save the final output
    cleaned_df.to_csv(final_output_with_brands_path, index=False)

    # Generate filtered outputs
    small_items_df = cleaned_df[
        cleaned_df["UBICAZIONE"].str.startswith(("A.", "B.", "C."))
    ]
    large_items_df = cleaned_df[
        ~cleaned_df["UBICAZIONE"].str.startswith(("A.", "B.", "C."))
    ]

    small_items_df.to_csv(output_csv_small_items_path, index=False)
    large_items_df.to_csv(output_csv_large_items_path, index=False)

    print(f"Final file saved at {final_output_with_brands_path}")
    print(f"Small items file saved at {output_csv_small_items_path}")
    print(f"Large items file saved at {output_csv_large_items_path}")


# Define paths for files
articles_file_path = "Z:/My Drive/rcs/Web/Tulero/app/data/mgs010_Stampa Anagrafica Articoli al 21-08 con upa e giac.xls"
old_oems_folder = "Z:/My Drive/rcs/Web/Tulero/app/data/oems"
brands_file_path = "Z:/My Drive/rcs/Web/Tulero/app/data/BRANDS.csv"
warehouse_location_file = "Z:/My Drive/rcs/Web/Tulero/app/data/mgs010_Stampa Anagrafica Articoli al 21-08 con ubi e giac.xls"
final_output_with_brands_path = (
    "Z:/My Drive/rcs/Web/Tulero/app/output/final_output_with_brands_path.csv"
)
output_csv_small_items_path = (
    "Z:/My Drive/rcs/Web/Tulero/app/output/output_csv_small_items_path.csv"
)
output_csv_large_items_path = (
    "Z:/My Drive/rcs/Web/Tulero/app/output/output_csv_large_items_path.csv"
)
warehouse_unknown_location_path = (
    "Z:/My Drive/rcs/Web/Tulero/app/output/warehouse_unknown_location_path.csv"
)


# Main Processing Call
process_files_with_location(
    articles_file_path,
    output_csv_small_items_path,
    old_oems_folder,
    brands_file_path,
    warehouse_location_file,
    final_output_with_brands_path,
    output_csv_small_items_path,
    output_csv_large_items_path,
    warehouse_unknown_location_path,
)


Validating sheets: 100%|██████████| 5/5 [00:00<00:00, 10.93it/s]


Number of columns before renaming: 5
Column names before renaming: ['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5']


Updating brands: 100%|██████████| 31915/31915 [00:00<00:00, 3190133.99it/s]


Final file saved at Z:/My Drive/rcs/Web/Tulero/app/output/final_output_with_brands_path.csv
Small items file saved at Z:/My Drive/rcs/Web/Tulero/app/output/output_csv_small_items_path.csv
Large items file saved at Z:/My Drive/rcs/Web/Tulero/app/output/output_csv_large_items_path.csv


In [6]:
import pandas as pd
from tqdm import tqdm


def is_unnamed(header):
    return str(header).startswith("Unnamed")


def validate_first_sheet(df):
    return (
        is_unnamed(df.columns[1])
        and is_unnamed(df.columns[2])
        and is_unnamed(df.columns[3])
        and is_unnamed(df.columns[4])
        and is_unnamed(df.columns[5])
        and not pd.isna(df.columns[0])
        and not pd.isna(df.columns[6])
    )


def validate_other_sheet(df):
    return (
        df.shape[1] == 6
        and not is_unnamed(df.columns[1])
        and not is_unnamed(df.columns[2])
        and not is_unnamed(df.columns[3])
        and not is_unnamed(df.columns[4])
        and is_unnamed(df.columns[0])
    )


def load_and_clean_warehouse_file(
    warehouse_location_file,
    output_csv_small_items_path,
    output_csv_large_items_path,
    warehouse_unknown_location_path,
):
    xls = pd.ExcelFile(warehouse_location_file)
    relevant_sheets = []
    first_sheet_validated = False

    for sheet_name in tqdm(xls.sheet_names, desc="Validating sheets"):
        df = pd.read_excel(xls, sheet_name=sheet_name, header=0, dtype=str)
        if not first_sheet_validated:
            if validate_first_sheet(df):
                relevant_sheets.append((sheet_name, df))
                first_sheet_validated = True
            else:
                raise ValueError("First sheet is not valid")
        else:
            if validate_other_sheet(df):
                relevant_sheets.append((sheet_name, df))

    if not relevant_sheets:
        raise ValueError("No relevant sheets found in the Excel file")

    # Identify and drop the column that starts with 'mgs'
    first_sheet_df = relevant_sheets[0][1]
    mgs_column = [col for col in first_sheet_df.columns if col.startswith("mgs")]
    if mgs_column:
        first_sheet_df = first_sheet_df.drop(columns=mgs_column)
    else:
        raise ValueError("No column starting with 'mgs' found")

    aligned_sheets = [first_sheet_df]
    for sheet_name, df in relevant_sheets[1:]:
        df.columns = first_sheet_df.columns
        aligned_sheets.append(df)

    df_combined = pd.concat(aligned_sheets, ignore_index=True)
    df_combined = df_combined[~df_combined.iloc[:, 2].isin(["", "."])]

    # Print current csv.
    df_combined.to_csv(cleaned_original_csv_path, index=False)

    # Drop the 'STAMPA ANAGRAFICA ARTICOLI' column
    df_combined = df_combined.drop(columns=["STAMPA ANAGRAFICA ARTICOLI"])

    # Check the number of columns before renaming
    print(f"Number of columns before renaming: {len(df_combined.columns)}")
    print(f"Column names before renaming: {df_combined.columns.tolist()}")

    # Assign column names based on the number of columns
    if len(df_combined.columns) == 5:
        df_combined.columns = [
            "CODICE PRODOTTO",
            "MARCA",
            "DESCRIZIONE",
            "UBICAZIONE",
            "GIACENZA",
        ]
    else:
        raise ValueError(f"Unexpected number of columns: {len(df_combined.columns)}")

    # Ensure specific columns are treated as strings
    df_combined["CODICE PRODOTTO"] = df_combined["CODICE PRODOTTO"].astype(str)
    df_combined["MARCA"] = df_combined["MARCA"].astype(str)
    df_combined["DESCRIZIONE"] = df_combined["DESCRIZIONE"].astype(str)
    # Replace NaN and empty strings in 'UBICAZIONE' with "Location Unknown"
    df_combined["UBICAZIONE"] = (
        df_combined["UBICAZIONE"]
        .fillna("Location Unknown")
        .replace("", "Location Unknown")
    )

    df_combined["GIACENZA"] = pd.to_numeric(
        df_combined["GIACENZA"].str.replace(",", "."), errors="coerce"
    )

    # Define patterns for valid small items and valid large items
    small_item_patterns = [
        r"^[A-Ca-c](?:\.|[0-9])",  # Existing pattern for 'A.', 'B.', 'C.', etc.
        r"^MAG S",  # Match 'MAG S'
        r"^SC[0-9]+",  # Match 'SC' followed by a number
        r"^SCAT(?:[0-9]+|\s)",  # Match 'SCAT' followed by a number or space
        r"^SCATOLO(?:[0-9]+|\s)",  # Match 'SCATOLO' followed by a number or space
        r"^TOTEM LEMA$",  # Match 'TOTEM LEMA'
        r"^V-[0-9]+",  # Match 'V-' followed by a number
    ]

    large_item_patterns = [
        r"^[D-Zd-z](?:\.|[0-9])",
        r"^2\.",  # Match '2.' at the beginning
        r"COSPEL",  # Match any string containing 'COSPEL'
        r"PIAZZALE",  # Match any string containing 'COSPEL'
        r"MAG\.ZF",  # Match 'MAG.ZF'
        r"^MG\.[0-9]+",  # Match 'MG.' followed by a number
        r"^MG\.[A-Za-z]",  # Match 'MG.' followed by a letter
        r"^Mg\.[A-Za-z]",  # Match 'Mg.' followed by a letter
        r"^mg\.[A-Za-z]",  # Match 'mg.' followed by a letter
        r"^ZF\.[A-Za-z]",  # Match 'ZF.' followed by a letter
    ]

    # Combine small item patterns into a single regex pattern
    valid_small_items_pattern = "|".join(small_item_patterns)
    valid_small_items = df_combined["UBICAZIONE"].str.match(valid_small_items_pattern)

    # Combine large item patterns into a single regex pattern
    valid_large_items_pattern = "|".join(large_item_patterns)
    valid_large_items = ~valid_small_items & df_combined["UBICAZIONE"].str.match(
        valid_large_items_pattern
    )

    # Define unknown items as those that do not match any small or large item patterns
    unknown_items = ~df_combined["UBICAZIONE"].str.match(
        f"{valid_small_items_pattern}|{valid_large_items_pattern}"
    )

    # Filter the DataFrame based on these patterns
    small_items_df = df_combined[valid_small_items]
    large_items_df = df_combined[valid_large_items]
    unknown_df = df_combined[unknown_items]

    # Identify and drop rows where the headers are repeated in the data
    unknown_df = unknown_df[unknown_df["CODICE PRODOTTO"] != "CODICE"]

    # Output the rows with unknown locations
    unknown_df.to_csv(warehouse_unknown_location_path, index=False)
    print(f"Unknown items saved to: {warehouse_unknown_location_path}")

    # Output the small and large items
    small_items_df.to_csv(output_csv_small_items_path, index=False)
    large_items_df.to_csv(output_csv_large_items_path, index=False)
    print(f"Small items saved to: {output_csv_small_items_path}")
    print(f"Large items saved to: {output_csv_large_items_path}")

    return small_items_df, large_items_df, unknown_df


# Paths
warehouse_location_file = "Z:/My Drive/rcs/Web/Tulero/app/data/mgs010_Stampa Anagrafica Articoli al 21-08 con ubi e giac.xls"
output_csv_small_items_path = (
    "Z:/My Drive/rcs/Web/Tulero/app/output/output_csv_small_items_path.csv"
)
output_csv_large_items_path = (
    "Z:/My Drive/rcs/Web/Tulero/app/output/output_csv_large_items_path.csv"
)
warehouse_unknown_location_path = (
    "Z:/My Drive/rcs/Web/Tulero/app/output/warehouse_unknown_location_path.csv"
)
cleaned_original_csv_path = "Z:/My Drive/rcs/Web/Tulero/app/output/cleaned_original.csv"

# Execute function
small_items_df, large_items_df, unknown_df = load_and_clean_warehouse_file(
    warehouse_location_file,
    output_csv_small_items_path,
    output_csv_large_items_path,
    warehouse_unknown_location_path,
)

# View the results
print("Small Items:")
print(small_items_df.head())

print("\nLarge Items:")
print(large_items_df.head())

print("\nUnknown Items:")
print(unknown_df.head())


Validating sheets: 100%|██████████| 5/5 [00:01<00:00,  4.91it/s]


Number of columns before renaming: 5
Column names before renaming: ['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5']
Unknown items saved to: Z:/My Drive/rcs/Web/Tulero/app/output/warehouse_unknown_location_path.csv
Small items saved to: Z:/My Drive/rcs/Web/Tulero/app/output/output_csv_small_items_path.csv
Large items saved to: Z:/My Drive/rcs/Web/Tulero/app/output/output_csv_large_items_path.csv
Small Items:
   CODICE PRODOTTO MARCA                     DESCRIZIONE    UBICAZIONE  \
3        0.001.003   RCS             TESTINA SOSP ANT SX  B.07.013.009   
6        0.001.176   RCS        INTERRUTTORE VOLVO TRUCK  C.12.016.030   
9        0.001.302   RCS  SENSORE GIRI MOTORE MITSUBISHI  C.33.004.035   
12       0.002.005   RCS       ELETTROV TUBO GASOLIO 12V  B.06.034.004   
13   0.002.005/24V   RCS       ELETTROV TUBO GASOLIO 24V  C.26.015.018   

    GIACENZA  
3        1.0  
6        1.0  
9        0.0  
12       1.0  
13       2.0  

Large Items:
    CODICE PRODOTT

In [7]:
import re
import pandas as pd
import os
from tqdm import tqdm

IGNORED_BRANDS = [
    "AP",
    "AREXONS",
    "ASSO",
    "ATE",
    "ATECSO",
    "AUTOCLIMA",
    "BIRTH",
    "BOSCH",
    "BREMBO",
    "BUGATTI",
    "CASCO",
    "CASTROL",
    "CEI",
    "CORTECO",
    "COVIND",
    "DAF",
    "DAYCO",
    "DELPHI",
    "DENSO",
    "DOLZ",
    "ELRING",
    "EMMERRE",
    "ERA",
    "EXIDE",
    "FAG",
    "FEBI",
    "FERODO",
    "FIAT",
    "FORD",
    "FRAP",
    "FTE",
    "GATES",
    "HELLA",
    "HOFFER",
    "IMASAF",
    "INA",
    "ISUZU",
    "IVECO",
    "JAPANPARTS",
    "JAPKO",
    "KNECHT",
    "KRIOS",
    "LEMFORDER",
    "LUK",
    "MAHLE",
    "MAN",
    "METELLI",
    "MEYLE",
    "MOBIL",
    "MONROE",
    "MOOG",
    "MULLER FILTER",
    "NISSAN",
    "NISSENS",
    "NK",
    "NRF",
    "OLSA",
    "OMP",
    "PEUGEOT",
    "PIAGGIO",
    "PIERBURG",
    "RAICAM",
    "RENAULT",
    "SACHS",
    "SCANIA",
    "SELENIA",
    "SIDAT",
    "SKF",
    "TEXTAR",
    "TRW",
    "TUDOR",
    "UFI",
    "VALEO",
    "VEMA",
    "VITAL SUSPENSIONS",
    "VOLVO",
    "VOLKSWAGEN",
    "ZETA-ERRE",
    "ZF",
]


def is_unnamed(header):
    return str(header).startswith("Unnamed")


def validate_first_sheet(df):
    return (
        is_unnamed(df.columns[1])
        and is_unnamed(df.columns[2])
        and is_unnamed(df.columns[3])
        and is_unnamed(df.columns[4])
        and is_unnamed(df.columns[5])
        and not pd.isna(df.columns[0])
        and not pd.isna(df.columns[6])
    )


def validate_other_sheet(df):
    return (
        df.shape[1] == 6
        and not is_unnamed(df.columns[1])
        and not is_unnamed(df.columns[2])
        and not is_unnamed(df.columns[3])
        and not is_unnamed(df.columns[4])
        and is_unnamed(df.columns[0])
    )


def get_oem_number(row, oem_lookup, IGNORED_BRANDS):
    if row["BRAND"] in IGNORED_BRANDS:
        return ""  # Skip filling OE code for ignored brands
    key = (row["CODICE PRODOTTO"], row["BRAND"][:5])
    return " | ".join(oem_lookup[key]) if key in oem_lookup else "Unknown OE"


def find_additional_cross_codes(codice_prodotto, padded_oe, cleaned_df, IGNORED_BRANDS):
    matches = cleaned_df[
        (cleaned_df["CODICE OE"] != "Unknown OE")
        & (~cleaned_df["BRAND"].isin(IGNORED_BRANDS))
    ]
    exact_matches = matches[
        matches["padded_oe"].str.contains(f" {codice_prodotto} ", regex=False)
    ]
    if not exact_matches.empty:
        return " | ".join(exact_matches["CODICE PRODOTTO"].unique())
    return ""


def update_brands(df_output, brands_file_path):
    # Load the brands file
    brands_df = pd.read_csv(brands_file_path, dtype=str)
    brands_df["Brand"] = brands_df["Brand"].astype(str).str.strip()
    brands_df["Match"] = brands_df["Match"].astype(str).str.strip()

    # Create a lookup dictionary from the brands file
    brand_lookup = dict(zip(brands_df["Brand"], brands_df["Match"]))

    # Update the BRANDS in the output dataframe
    tqdm.pandas(desc="Updating brands")
    df_output["BRAND"] = df_output["BRAND"].progress_apply(
        lambda x: brand_lookup.get(x, x)
    )
    return df_output


def optimized_cross_code_generation(cleaned_df, ignored_brands):
    cross_references = {}
    for _, row in tqdm(
        cleaned_df.iterrows(), total=len(cleaned_df), desc="Generating cross codes"
    ):
        codice_oe = row["CODICE OE"]
        codice_prodotto = row["CODICE PRODOTTO"]
        brand = row["BRAND"]

        if codice_oe != "Unknown OE" and brand not in ignored_brands:
            if codice_oe not in cross_references:
                cross_references[codice_oe] = []
            cross_references[codice_oe].append(codice_prodotto)

    cross_codes_series = pd.Series(index=cleaned_df.index, dtype=str)
    for codice_oe, prodotti in tqdm(
        cross_references.items(), desc="Updating CODICI CROSS"
    ):
        cross_codes = {
            prodotto: " | ".join([code for code in prodotti if code != prodotto])
            for prodotto in prodotti
        }
        cross_codes_series.update(pd.Series(cross_codes))

    return cross_codes_series.fillna("")


def load_and_clean_warehouse_file(warehouse_location_file):
    xls = pd.ExcelFile(warehouse_location_file)
    relevant_sheets = []
    first_sheet_validated = False

    for sheet_name in tqdm(xls.sheet_names, desc="Validating sheets"):
        df = pd.read_excel(xls, sheet_name=sheet_name, header=0, dtype=str)
        if not first_sheet_validated:
            if validate_first_sheet(df):
                relevant_sheets.append((sheet_name, df))
                first_sheet_validated = True
            else:
                raise ValueError("First sheet is not valid")
        else:
            if validate_other_sheet(df):
                relevant_sheets.append((sheet_name, df))

    if not relevant_sheets:
        raise ValueError("No relevant sheets found in the Excel file")

    # Identify and drop the column that starts with 'mgs'
    first_sheet_df = relevant_sheets[0][1]
    mgs_column = [col for col in first_sheet_df.columns if col.startswith("mgs")]
    if mgs_column:
        first_sheet_df = first_sheet_df.drop(columns=mgs_column)

    aligned_sheets = [first_sheet_df]
    for sheet_name, df in relevant_sheets[1:]:
        df.columns = first_sheet_df.columns
        aligned_sheets.append(df)

    df_combined = pd.concat(aligned_sheets, ignore_index=True)
    df_combined = df_combined[~df_combined.iloc[:, 2].isin(["", "."])]

    # Drop unnecessary column
    df_combined = df_combined.drop(columns=["STAMPA ANAGRAFICA ARTICOLI"])

    # Assign column names based on the number of columns
    if len(df_combined.columns) == 5:
        df_combined.columns = [
            "CODICE PRODOTTO",
            "MARCA",
            "DESCRIZIONE",
            "UBICAZIONE",
            "GIACENZA",
        ]
    else:
        raise ValueError(f"Unexpected number of columns: {len(df_combined.columns)}")

    # Ensure specific columns are treated as strings
    df_combined["CODICE PRODOTTO"] = df_combined["CODICE PRODOTTO"].astype(str)
    df_combined["MARCA"] = df_combined["MARCA"].astype(str)
    df_combined["DESCRIZIONE"] = df_combined["DESCRIZIONE"].astype(str)
    df_combined["UBICAZIONE"] = (
        df_combined["UBICAZIONE"]
        .fillna("Location Unknown")
        .replace("", "Location Unknown")
    )
    df_combined["GIACENZA"] = pd.to_numeric(
        df_combined["GIACENZA"].str.replace(",", "."), errors="coerce"
    )

    # Keep only small items (A, B, C)
    pattern = r"^[A-Ca-c](?:\.|[0-9])"
    valid_small_items = df_combined["UBICAZIONE"].str.match(pattern)
    df_combined = df_combined[valid_small_items]

    return df_combined


def process_files(
    articles_file_path,
    output_file_path,
    old_oems_folder,
    brands_file_path,
    warehouse_location_file,
    final_output_with_brands_path,
    update_progress=None,
):
    # Load and clean the warehouse file
    warehouse_df = load_and_clean_warehouse_file(warehouse_location_file)

    # Load and clean the articles file

    xls = pd.ExcelFile(articles_file_path)
    relevant_sheets = []
    first_sheet_validated = False

    for sheet_name in tqdm(xls.sheet_names, desc="Validating sheets"):
        df = pd.read_excel(
            xls, sheet_name=sheet_name, header=0, dtype=str
        )  # Ensure all columns are read as strings
        if not first_sheet_validated:
            if validate_first_sheet(df):
                relevant_sheets.append((sheet_name, df))
                first_sheet_validated = True
            else:
                raise ValueError("First sheet is not valid")
        else:
            if validate_other_sheet(df):
                relevant_sheets.append((sheet_name, df))

    if not relevant_sheets:
        raise ValueError("No relevant sheets found in the Excel file")

    first_sheet_df = relevant_sheets[0][1]
    mgs_column = [col for col in first_sheet_df.columns if col.startswith("mgs")]
    if mgs_column:
        first_sheet_df = first_sheet_df.drop(columns=mgs_column)

    aligned_sheets = [first_sheet_df]
    for sheet_name, df in relevant_sheets[1:]:
        df.columns = first_sheet_df.columns
        aligned_sheets.append(df)

    df_combined = pd.concat(aligned_sheets, ignore_index=True)
    df_combined = df_combined[~df_combined.iloc[:, 2].isin(["", "."])]
    df_combined = df_combined.drop(columns=["STAMPA LISTINI"])

    df_combined.columns = [
        "CODICE PRODOTTO",
        "BRAND",
        "DESCRIZIONE",
        "GIACENZA",
        "PRZ. ULT. ACQ.",
    ]

    for col in [
        "CODICE OE",
        "CODICI CROSS",
        "LINK IMMAGINE",
        "CATEGORIA",
        "SCHEDA TECNICA",
        "SCHEDA DI SICUREZZA",
        "CONFEZIONE",
        "QUANTITÀ MINIMA",
        "META.LUNGHEZZA",
        "META.LARGHEZZA",
        "META.PROFONDITA'",
        "META. ...",
        "UBICAZIONE",
    ]:
        df_combined[col] = pd.NA

    # Ensure specified columns are treated as strings
    columns_to_ensure_as_strings = [
        "CODICE PRODOTTO",
        "BRAND",
        "DESCRIZIONE",
        "CODICE OE",
        "CODICI CROSS",
        "LINK IMMAGINE",
        "CATEGORIA",
        "SCHEDA TECNICA",
        "SCHEDA DI SICUREZZA",
        "CONFEZIONE",
        "META.LUNGHEZZA",
        "META.LARGHEZZA",
        "META.PROFONDITA'",
        "META. ...",
        "UBICAZIONE",
    ]
    df_combined[columns_to_ensure_as_strings] = df_combined[
        columns_to_ensure_as_strings
    ].astype(str)

    df_combined["GIACENZA"] = pd.to_numeric(
        df_combined["GIACENZA"].str.replace(",", "."), errors="coerce"
    )
    df_combined["PRZ. ULT. ACQ."] = pd.to_numeric(
        df_combined["PRZ. ULT. ACQ."].str.replace(",", "."), errors="coerce"
    )

    df_combined = df_combined[df_combined["GIACENZA"] > 0]
    df_combined = df_combined[df_combined["PRZ. ULT. ACQ."].notna()]

    # Merge with warehouse data
    df_combined = df_combined.merge(
        warehouse_df[["CODICE PRODOTTO", "MARCA", "UBICAZIONE"]],
        left_on=["CODICE PRODOTTO", "BRAND"],
        right_on=["CODICE PRODOTTO", "MARCA"],
        how="left",
    )

    df_combined = df_combined.drop(columns=["MARCA"])

    # Assign shipping category based on UBICAZIONE
    df_combined["SHIPPING_CAT"] = df_combined["UBICAZIONE"].apply(
        lambda x: "9.90" if pd.notna(x) and re.match(r"^[A-Ca-c]", x) else "UNKNOWN"
    )

    # Drop the UBICAZIONE column since it's no longer needed
    df_combined = df_combined.drop(columns=["UBICAZIONE"])

    # No need to save and reload, continue with the DataFrame in memory
    cleaned_df = df_combined.copy()
    cleaned_df["CODICE PRODOTTO"] = cleaned_df["CODICE PRODOTTO"].str.strip()
    cleaned_df["CODICE OE"] = cleaned_df["CODICE OE"].str.strip()
    cleaned_df["BRAND"] = cleaned_df["BRAND"].str.strip()

    old_oems_files = [
        file
        for file in os.listdir(old_oems_folder)
        if file.startswith("oemsDC") and file.endswith(".csv")
    ]
    all_oem_mappings = pd.DataFrame()

    for file_name in tqdm(old_oems_files, desc="Building old OEM mappings"):
        file_path = os.path.join(old_oems_folder, file_name)
        oems_df = pd.read_csv(file_path, dtype=str)
        oems_df["article_altc"] = oems_df["article_altc"].astype(str).str.strip()
        oems_df["oem_number"] = (
            oems_df["oem_number"].astype(str).str.strip().str.replace(" ", "")
        )
        oems_df["article_alt_brands"] = (
            oems_df["article_alt_brands"].astype(str).str.strip()
        )
        oems_df["brand_prefix"] = oems_df["article_alt_brands"].str[:5]
        all_oem_mappings = pd.concat(
            [all_oem_mappings, oems_df[["article_altc", "oem_number", "brand_prefix"]]]
        )

    oem_lookup = (
        all_oem_mappings.groupby(["article_altc", "brand_prefix"])["oem_number"]
        .apply(list)
        .to_dict()
    )

    # Update CODICE OE using the get_oem_number function
    tqdm.pandas(desc="Updating CODICE OE with old OEMs")
    cleaned_df["CODICE OE"] = cleaned_df.progress_apply(
        lambda row: get_oem_number(row, oem_lookup, IGNORED_BRANDS), axis=1
    )

    # Update PREZZO based on PRZ. ULT. ACQ.
    cleaned_df["PRZ. ULT. ACQ."] = pd.to_numeric(
        cleaned_df["PRZ. ULT. ACQ."], errors="coerce"
    )
    cleaned_df["PREZZO"] = cleaned_df["PRZ. ULT. ACQ."].apply(
        lambda x: round(x * 1.25, 2) if pd.notnull(x) else x
    )
    cleaned_df["SHIPPING_CAT"] = pd.to_numeric(
        cleaned_df["SHIPPING_CAT"], errors="coerce"
    )
    cleaned_df = cleaned_df.drop(columns=["PRZ. ULT. ACQ."])

    # Ensure specified columns are treated as strings before reordering
    cleaned_df[columns_to_ensure_as_strings] = cleaned_df[
        columns_to_ensure_as_strings
    ].astype(str)

    # Reorder columns
    columns_order = [
        "CODICE PRODOTTO",
        "CODICE OE",
        "CODICI CROSS",
        "BRAND",
        "DESCRIZIONE",
        "LINK IMMAGINE",
        "CATEGORIA",
        "GIACENZA",
        "PREZZO",
        "SCHEDA TECNICA",
        "SCHEDA DI SICUREZZA",
        "CONFEZIONE",
        "QUANTITÀ MINIMA",
        "SHIPPING_CAT",  # Include SHIPPING_CAT
        "META.LUNGHEZZA",
        "META.LARGHEZZA",
        "META.PROFONDITA'",
        "META. ...",
    ]
    cleaned_df = cleaned_df[columns_order]
    cleaned_df["CODICI CROSS"] = ""

    # Apply optimized cross-code generation function
    cleaned_df["CODICI CROSS"] = optimized_cross_code_generation(
        cleaned_df, IGNORED_BRANDS
    )

    # Handle cases where CODICE OE is unknown and brand is not ignored
    cleaned_df["padded_oe"] = " " + cleaned_df["CODICE OE"].str.strip() + " "
    unknown_oe_mask = (cleaned_df["CODICE OE"] == "Unknown OE") & (
        ~cleaned_df["BRAND"].isin(IGNORED_BRANDS)
    )
    cleaned_df.loc[unknown_oe_mask, "CODICI CROSS"] = cleaned_df.loc[
        unknown_oe_mask, "CODICE PRODOTTO"
    ].progress_apply(
        lambda codice_prodotto: find_additional_cross_codes(
            codice_prodotto, cleaned_df["padded_oe"], cleaned_df, IGNORED_BRANDS
        )
    )

    # Drop the 'padded_oe' column
    cleaned_df = cleaned_df.drop(columns=["padded_oe"])

    # Fill the "CONFEZIONE" column with "1 pz" and the "QUANTITÀ MINIMA" column with "1"
    cleaned_df["CONFEZIONE"] = "1 pz"
    cleaned_df["QUANTITÀ MINIMA"] = "1"

    # Set CATEGORIA to "Ricambio"
    cleaned_df["CATEGORIA"] = "Ricambio"

    # Ensure specified columns remain empty
    columns_to_keep_empty = [
        "LINK IMMAGINE",
        "SCHEDA TECNICA",
        "SCHEDA DI SICUREZZA",
        "META.LUNGHEZZA",
        "META.LARGHEZZA",
        "META.PROFONDITA'",
        "META. ...",
    ]
    cleaned_df[columns_to_keep_empty] = ""

    # Step 4: Update brands
    cleaned_df = update_brands(cleaned_df, brands_file_path)

    # After updating brands, set CODICE OE and CODICI CROSS to empty for ignored brands
    ignored_brands_mask = cleaned_df["BRAND"].isin(IGNORED_BRANDS)
    cleaned_df.loc[ignored_brands_mask, ["CODICE OE", "CODICI CROSS"]] = ""

    # Save the final output directly
    cleaned_df.to_csv(final_output_with_brands_path, index=False)
    print(f"Brands updated and final file saved at {final_output_with_brands_path}")
